In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from glob import glob 
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import pickle
from skimage import feature
import os
import pandas as pd
from collections import Counter
from itertools import chain
from sklearn.decomposition import KernelPCA
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision


In [2]:
file_path = 'Labels/projectLabelsRaw.csv'
labels = pd.read_csv(file_path)
root = 'Videos'

file_names = sorted(glob(root+"/*.webm"))
labels.head()

,annotation_id,annotator,created_at,id,lead_time,tricks,updated_at,url
0,1,1,2024-10-10T05:22:48.696876Z,1,51.152,"[{""start"":0.00703674832962138,""end"":4.07075890...",2024-10-10T05:22:48.696894Z,/data/upload/1/53888dc4-img-6368_2160p_25fps_1...
1,2,1,2024-10-10T05:24:24.742398Z,2,95.846,"[{""start"":0.7558730512249444,""end"":1.329608017...",2024-10-10T05:24:24.742417Z,/data/upload/1/0247b9d2-img-6369_1080p_25fps_1...
2,3,1,2024-10-10T05:25:53.957437Z,3,89.028,"[{""start"":5.656628619153675,""end"":6.4680066815...",2024-10-10T05:25:53.957457Z,/data/upload/1/a0686400-img-6370_1080p_25fps_1...
3,4,1,2024-10-10T05:26:43.707072Z,4,49.552,"[{""start"":1.9139955456570157,""end"":2.923768930...",2024-10-10T05:26:43.707090Z,/data/upload/1/91c7ef02-img-6371_1080p_25fps_1...
4,6,1,2024-10-14T02:26:48.192917Z,5,171.526,"[{""start"":0.6705745967741936,""end"":2.826729838...",2024-10-14T02:26:48.192935Z,/data/upload/1/96a0206d-1-hecho-con-clipchamp_...


In [3]:
fps = 25


# Función para convertir segundos a frames
def convert_to_frames(tricks, fps):
    tricks_dicts = eval(tricks)
    for trick in tricks_dicts:
        trick['start_frame'] = int(trick['start'] * fps)
        trick['end_frame'] = int(trick['end'] * fps)
    return tricks_dicts

# Aplicamos la conversión a todas las filas
labels['tricks_in_frames'] = labels['tricks'].apply(lambda x: convert_to_frames(x, fps))

# Mostramos las primeras filas con las etiquetas convertidas a frames
labels[['tricks', 'tricks_in_frames']].head()

,tricks,tricks_in_frames
0,"[{""start"":0.00703674832962138,""end"":4.07075890...","[{'start': 0.00703674832962138, 'end': 4.07075..."
1,"[{""start"":0.7558730512249444,""end"":1.329608017...","[{'start': 0.7558730512249444, 'end': 1.329608..."
2,"[{""start"":5.656628619153675,""end"":6.4680066815...","[{'start': 5.656628619153675, 'end': 6.4680066..."
3,"[{""start"":1.9139955456570157,""end"":2.923768930...","[{'start': 1.9139955456570157, 'end': 2.923768..."
4,"[{""start"":0.6705745967741936,""end"":2.826729838...","[{'start': 0.6705745967741936, 'end': 2.826729..."


In [4]:
labels.to_csv("Labels/UpdatedLables.csv", index=False)

In [5]:
labelsNew = list()
labels_by_frames = list()
last_end = 0
trfr = labels['tricks_in_frames']
for n in range(len(trfr)):
    file_name = file_names[n]
    video = cv2.VideoCapture(file_name)  
    numframes = 0
    while(video.isOpened()):
        ret, f = video.read()
        numframes+=1
        if not ret:
            break
    f = 0
    video.release()
    tricks = trfr[n]
    labels_by_frames.append([])
    true_len = numframes
    for trick in range(len(tricks)):
        start = tricks[trick]['start_frame']
        end = tricks[trick]['end_frame']
        labelsl = tricks[trick]['labels']
        
        if(start > last_end + 1):
            labels_by_frames[n].append({
            'start_frame': last_end +1,
            'end_frame': start,
            'labels': None
            })

        labels_by_frames[n].append({
            'start_frame': start,
            'end_frame': end,
            'labels': labelsl
        })

        if(trick == len(tricks)-1 and end < true_len):

            labels_by_frames[n].append({
                'start_frame': end,
                'end_frame': true_len,
                'labels': None
            })
        last_end = end




In [6]:
print(labels_by_frames[0])

[{'start_frame': 0, 'end_frame': 101, 'labels': ['walking_to_camera']}, {'start_frame': 101, 'end_frame': 157, 'labels': ['walking_away']}, {'start_frame': 157, 'end_frame': 461, 'labels': None}]


In [7]:
max_frames = []
for n in range(len(labels_by_frames)):
   a = []
   max_frames.append(max([trick['end_frame'] for trick in labels_by_frames[n]]))

frames_labels = []
# Inicializamos una lista donde cada índice representa un frame
for max_frame in max_frames:
   frames_labels.append([None] * (max_frame + 1)) # +1 para incluir el último frame

# Rellenamos los frames con sus etiquetas según los rangos de start_frame y end_frame
for n in range(len(frames_labels)):
   for idx, trick in enumerate(labels_by_frames[n]):
         for frame in range(trick['start_frame'], trick['end_frame'] + 1):
            frames_labels[n][frame] = trick['labels']


frames_labels[0][:10]

[['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera']]

In [8]:

groups = []
labels = []
dic_clases = {}
file_tipe = ""

for n in frames_labels:
    print(len(n))


462
323
287
329
384
301
319
325
307
310
295
307
255
492
621
380


In [9]:
def draw_landmarks_on_image(rgb_image, detection_result):
  pose_landmarks_list = detection_result.pose_landmarks
  annotated_image = np.copy(rgb_image)


  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]


    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_ladnmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
  return annotated_image

In [69]:
complete_frame_landmarks = list()
for h  in tqdm(range(len(file_names)-1)):
    groups = []
    labels = []
    file_name = file_names[h]
    video = cv2.VideoCapture(file_name)  
    frames = []
    while(video.isOpened()):
        ret, f = video.read()
        if(f is not None):
            #f = cv2.resize(f,(500,500)) 
            frames.append(f)
        else:
            video.release()
            break
    
    base_options = python.BaseOptions(model_asset_path='pose_landmarker_lite.task')
    options = vision.PoseLandmarkerOptions(
        base_options=base_options,
        output_segmentation_masks=True)
    detector = vision.PoseLandmarker.create_from_options(options)

    for frame in frames:
        if frame.shape[-1] == 3:  
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        else:
            frame_rgb = frame  
        

        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)


        detection_result = detector.detect(mp_image)

        if detection_result.pose_landmarks:
            
            
            for i, landmarkl in enumerate(detection_result.pose_landmarks):
                frame_landmarks = list()
                for n in range(len(landmarkl)):
                    frame_data = dict()
                    landmark = landmarkl[n]
                    x = landmark.x
                    y = landmark.y
                    z = landmark.z
                    frame_data[f'landmark_{n}_x'] = x
                    frame_data[f'landmark_{n}_y'] = y
                    frame_data[f'landmark_{n}_z'] = z
                    frame_landmarks.append(frame_data.copy())
            complete_frame_landmarks.append(frame_landmarks.copy())
            
            # Añadir el diccionario a la lista
            
        
        
        
        #annotated_image = draw_landmarks_on_image(frame_rgb, detection_result)
        
        #cv2.imshow("Pose Detection", cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))

        #if cv2.waitKey(1) & 0xFF == ord('q'):
           # break
    
    
'''
    for n in range(len(complete_frame_landmarks )-5):

            group = []
            group_labels = []
            for i in range(n, n+5):
                group.append(complete_frame_landmarks[i])
                group_labels.append(frames_labels[h][i])
            

            flattened_labels = list(chain.from_iterable(
                label if label is not None else  [None]  
                for label in group_labels
            ))

            if flattened_labels:
                most_common_label = Counter(flattened_labels).most_common(1)[0][0]
            else:
                most_common_label = None 
            
            groups.append(group)
            labels.append(most_common_label)
    
    '''


  0%|          | 0/15 [00:00<?, ?it/s]I0000 00:00:1728880795.102271  330958 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1728880795.105420  365388 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: AMD Radeon RX 6650 XT (navi23, LLVM 15.0.7, DRM 3.42, 5.15.0-122-generic)
W0000 00:00:1728880795.160649  365389 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1728880795.174012  365392 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/home/camilin/Documents/Icesi/Inteligencia Artificial/Proyecto Final/ProyectoIA-2024/.venv/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabas

'\n    for n in range(len(complete_frame_landmarks )-5):\n\n            group = []\n            group_labels = []\n            for i in range(n, n+5):\n                group.append(complete_frame_landmarks[i])\n                group_labels.append(frames_labels[h][i])\n            \n\n            flattened_labels = list(chain.from_iterable(\n                label if label is not None else  [None]  \n                for label in group_labels\n            ))\n\n            if flattened_labels:\n                most_common_label = Counter(flattened_labels).most_common(1)[0][0]\n            else:\n                most_common_label = None \n            \n            groups.append(group)\n            labels.append(most_common_label)\n    \n    '

In [70]:
organized_frames=list()
for n in complete_frame_landmarks:
    thing = dict()
    for land  in n:
        for title in land:
            thing[title] = land[title]
    organized_frames.append(thing.copy())

organized_frames[0]

{'landmark_0_x': 0.4981614947319031,
 'landmark_0_y': 0.46747535467147827,
 'landmark_0_z': -0.10016521066427231,
 'landmark_1_x': 0.4986411929130554,
 'landmark_1_y': 0.44653165340423584,
 'landmark_1_z': -0.11698363721370697,
 'landmark_2_x': 0.49973252415657043,
 'landmark_2_y': 0.4455466568470001,
 'landmark_2_z': -0.1170855462551117,
 'landmark_3_x': 0.5007708668708801,
 'landmark_3_y': 0.4443967342376709,
 'landmark_3_z': -0.11714379489421844,
 'landmark_4_x': 0.49823063611984253,
 'landmark_4_y': 0.4488352835178375,
 'landmark_4_z': -0.10191246122121811,
 'landmark_5_x': 0.4988863170146942,
 'landmark_5_y': 0.44950196146965027,
 'landmark_5_z': -0.10197591036558151,
 'landmark_6_x': 0.49930766224861145,
 'landmark_6_y': 0.4503471553325653,
 'landmark_6_z': -0.10203175246715546,
 'landmark_7_x': 0.5130833983421326,
 'landmark_7_y': 0.45320361852645874,
 'landmark_7_z': -0.14518900215625763,
 'landmark_8_x': 0.5127291679382324,
 'landmark_8_y': 0.4592084288597107,
 'landmark_8_z':

In [71]:
print(frames_labels[0][0])

['walking_to_camera']


In [72]:
flattened_labels =list()

for n in frames_labels:
    for i in n:
        flattened_labels.append(i)

flattened_labels

[['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_camera'],
 ['walking_to_ca

In [73]:

for n in range(len(organized_frames)):
    organized_frames[n]['labels'] = flattened_labels[n]
    organized_frames[n]['frame'] = n

for n in organized_frames[0]:
    print(n)

landmark_0_x
landmark_0_y
landmark_0_z
landmark_1_x
landmark_1_y
landmark_1_z
landmark_2_x
landmark_2_y
landmark_2_z
landmark_3_x
landmark_3_y
landmark_3_z
landmark_4_x
landmark_4_y
landmark_4_z
landmark_5_x
landmark_5_y
landmark_5_z
landmark_6_x
landmark_6_y
landmark_6_z
landmark_7_x
landmark_7_y
landmark_7_z
landmark_8_x
landmark_8_y
landmark_8_z
landmark_9_x
landmark_9_y
landmark_9_z
landmark_10_x
landmark_10_y
landmark_10_z
landmark_11_x
landmark_11_y
landmark_11_z
landmark_12_x
landmark_12_y
landmark_12_z
landmark_13_x
landmark_13_y
landmark_13_z
landmark_14_x
landmark_14_y
landmark_14_z
landmark_15_x
landmark_15_y
landmark_15_z
landmark_16_x
landmark_16_y
landmark_16_z
landmark_17_x
landmark_17_y
landmark_17_z
landmark_18_x
landmark_18_y
landmark_18_z
landmark_19_x
landmark_19_y
landmark_19_z
landmark_20_x
landmark_20_y
landmark_20_z
landmark_21_x
landmark_21_y
landmark_21_z
landmark_22_x
landmark_22_y
landmark_22_z
landmark_23_x
landmark_23_y
landmark_23_z
landmark_24_x
landmark

In [74]:
df = pd.DataFrame(organized_frames)


In [75]:
df.to_csv('datos.csv', index=False)